## CONFIGURACION: 


*   Optim: Adam-8bit
*   Batch size: 1024 (16x64 grad_acc.)
*   LR: 1e-7*64
*   Weight decay: 0.0
*   Other:






**Ctrl**+Shift+i

```
function ClickConnect(){
console.log("Working"); 
document.querySelector("colab-toolbar-button#connect").click() 
}
setInterval(ClickConnect,60000)
```



En consola local:
pip install pynput

Dentro del inrerprete python:

```
from pynput.mouse import Controller, Button 
import time 

def CollabClickHack(num_seconds):
	mouse = Controller() 
	while True: 
		mouse.click(Button.left,1) 
		print('clicked') 
		time.sleep(num_seconds)
		
CollabClickHack(25)
```



In [ ]:
!nvidia-smi -L  

GPU 0: Tesla P100-PCIE-16GB (UUID: GPU-e239c673-34d6-af4c-fd11-06f6eed5cc4a)


In [ ]:
!pip install -qqq datasets folium==0.9.1 transformers[sentencepiece]==4.20.1 rouge-score==0.0.4 nltk wandb bitsandbytes-cuda111
!git config --global credential.helper storedChk
!apt -qqq install git-lfs
!apt -qqq  autoremove

     |████████████████████████████████| 4.0 MB 4.2 MB/s 


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
import wandb
os.environ["WANDB_API_KEY"]="fdd7781abf964d3e05353120f73a5c93e10dc616"
os.environ["WANDB_ENTITY"]="mia3_esg_tfm"
os.environ["WANDB_PROJECT"]="Google Pegasus-Newsroom-Summarization CNNDaily 50K"
os.environ["WANDB_START_METHOD"] = "thread"

wandb.init(project="Google Pegasus-Newsroom-Summarization CNNDaily 50K", entity="mia3_esg_tfm")

wandb: Currently logged in as: osmalo (mia3_esg_tfm). Use `wandb login --relogin` to force relogin


In [ ]:
import nltk
nltk.download('punkt')
import transformers

print(transformers.__version__)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


4.20.1


# Fine-tuning a model on a summarization task

In [ ]:
from transformers import AutoTokenizer
from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer

initial_model = 't5-base'
model_name = 't5-base-newsroom-cnn_50k-adam8bit-bs16x64acc'
model_id = 'oMateos2020/t5-base-cnn_50k-adam8bit-bs16x64acc'
model_dir = '/content/drive/Shareddrives/UCM_SHARED/TFM_ESG/t5-base-cnn_50k-adam8bit-bs16x64acc'

max_input_length = 512

tokenizer = AutoTokenizer.from_pretrained(initial_model, model_max_length=max_input_length) 
model = AutoModelForSeq2SeqLM.from_pretrained(initial_model, num_beams=2)

Downloading:   0%|          | 0.00/1.17k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/773k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.32M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/850M [00:00<?, ?B/s]

## Loading the Preprocessed Dataset

In [ ]:
from datasets import Dataset,DatasetDict, load_dataset, load_metric, load_from_disk

# Load preprocessed Data 
tokenized_datasets = load_from_disk(dataset_path='/content/drive/Shareddrives/UCM_SHARED/TFM_ESG/Preprocessed Datasets for training/T5_CNNDailyFull')
metric = load_metric("rouge")
tokenized_datasets.num_rows

# Recortamos DS
tokenized_datasets = DatasetDict({
    'train': Dataset.from_dict(tokenized_datasets['train'][:50000]),
    'test': Dataset.from_dict(tokenized_datasets['test'][:2500]),
    'validation': Dataset.from_dict(tokenized_datasets['validation'][:2500])})
tokenized_datasets.shuffle()
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 50000
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 2500
    })
    validation: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 2500
    })
})

## Fine-tuning the model

In [ ]:
batch_size = 16
gradient_accumulation_steps = 64

#https://huggingface.co/docs/transformers/v4.19.2/en/performance
args = Seq2SeqTrainingArguments(
    output_dir=model_name,  
    dataloader_num_workers=2,
    eval_steps=50, #280
    evaluation_strategy="steps", #epoch
    fp16=True,
    generation_num_beams=2, # En inferencia ya entrenado subirlo a 4
    greater_is_better=True,
    learning_rate=1e-7*gradient_accumulation_steps, # 1e-6*gradient_accumulation_steps
    logging_steps=50,#280
    logging_strategy="steps", #epoch
    metric_for_best_model="rouge1",
    max_steps = 300, 
    optim="adamw_hf", # Probar 8-bitAdam
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    predict_with_generate=True,
    push_to_hub=False,
    save_strategy="no", #epoch
    report_to="wandb",
    warmup_steps=100, 
    weight_decay=0.0,
    gradient_accumulation_steps = gradient_accumulation_steps,
    #eval_accumulation_steps = gradient_accumulation_steps,
    #gradient_checkpointing=True,
    label_smoothing_factor = 0.1,
    #group_by_length=False
)

data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [ ]:
#8-bit Adam: similar memory improvement as with Adafactor while keeping the full rolling average of the gradients

import bitsandbytes as bnb
from torch import nn
from transformers.trainer_pt_utils import get_parameter_names

decay_parameters = get_parameter_names(model, [nn.LayerNorm])
decay_parameters = [name for name in decay_parameters if "bias" not in name]
optimizer_grouped_parameters = [
    {
        "params": [p for n, p in model.named_parameters() if n in decay_parameters],
        "weight_decay": args.weight_decay,
    },
    {
        "params": [p for n, p in model.named_parameters() if n not in decay_parameters],
        "weight_decay": 0.0,
    },
]

optimizer_kwargs = {
    "betas": (args.adam_beta1, args.adam_beta2),
    "eps": args.adam_epsilon,
}
optimizer_kwargs["lr"] = args.learning_rate
adam_bnb_optim = bnb.optim.Adam8bit(
    optimizer_grouped_parameters,
    betas=(args.adam_beta1, args.adam_beta2),
    eps=args.adam_epsilon,
    lr=args.learning_rate,
)

==============================WARNING: DEPRECATED!==============================
WARNING! This version of bitsandbytes is deprecated. Please switch to `pip install bitsandbytes` and the new repo: https://github.com/TimDettmers/bitsandbytes
==============================WARNING: DEPRECATED!==============================


The last thing to define for our `Seq2SeqTrainer` is how to compute the metrics from the predictions. We need to define a function for this, which will just use the `metric` we loaded earlier, and we have to do a bit of pre-processing to decode the predictions into texts:

In [ ]:
import nltk
import numpy as np

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    
    # Rouge expects a newline after each sentence
    decoded_preds = ["\n".join(nltk.sent_tokenize(pred.strip())) for pred in decoded_preds]
    decoded_labels = ["\n".join(nltk.sent_tokenize(label.strip())) for label in decoded_labels]
    
    result = metric.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)
    # Extract a few results
    result = {key: value.mid.fmeasure * 100 for key, value in result.items()}
    
    # Add mean generated length
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)
    
    return {k: round(v, 4) for k, v in result.items()}

Optimizer & Scheduler


In [ ]:
from transformers import EarlyStoppingCallback

trainer = Seq2SeqTrainer(
    model,
    args,
    optimizers=(adam_bnb_optim, None),
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

max_steps is given, it will override any value given in num_train_epochs
Using cuda_amp half precision backend


In [ ]:
result = trainer.train()

***** Running training *****
  Num examples = 50000
  Num Epochs = 7
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 1024
  Gradient Accumulation steps = 64
  Total optimization steps = 300
Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"


Step,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len
50,5.977000,4.242215,24.888500,11.058300,20.301900,22.975000,18.998400
100,5.307000,4.218566,25.180900,11.316200,20.588500,23.254600,18.998400
150,5.446200,4.217928,25.171100,11.314900,20.582900,23.251400,18.998400
200,5.245700,4.217939,25.171100,11.314900,20.582900,23.251400,18.998400
250,5.837000,4.217920,25.171100,11.314900,20.582900,23.251400,18.998400
300,5.400900,4.217920,25.171100,11.314900,20.582900,23.251400,18.998400


***** Running Evaluation *****
  Num examples = 2500
  Batch size = 16
***** Running Evaluation *****
  Num examples = 2500
  Batch size = 16
***** Running Evaluation *****
  Num examples = 2500
  Batch size = 16
***** Running Evaluation *****
  Num examples = 2500
  Batch size = 16
***** Running Evaluation *****
  Num examples = 2500
  Batch size = 16
***** Running Evaluation *****
  Num examples = 2500
  Batch size = 16


Training completed. Do not forget to share your model on huggingface.co/models =)




In [ ]:
wandb.finish()

eval/gen_len,▁▁▁▁▁▁
eval/loss,█▁▁▁▁▁
eval/rouge1,▁█████
eval/rouge2,▁█████
eval/rougeL,▁█████
eval/rougeLsum,▁█████
eval/runtime,▁▁▃▄█▄
eval/samples_per_second,██▆▅▁▅
eval/steps_per_second,▇█▆▅▁▅
train/epoch,▁▁▂▂▄▄▅▅▇▇███
train/global_step,▁▁▂▂▄▄▅▅▇▇███


In [ ]:
print(result)

TrainOutput(global_step=300, training_loss=5.535631001790365, metrics={'train_runtime': 25787.8915, 'train_samples_per_second': 11.913, 'train_steps_per_second': 0.012, 'total_flos': 1.9017024172720128e+17, 'train_loss': 5.535631001790365, 'epoch': 6.25})


## Referencias

1.   https://huggingface.co/docs/transformers/v4.19.2/en/performance
2.   https://huggingface.co/datasets/cnn_dailymail

